In [ ]:
# 라이브러리 불러오기 및 함수화
import re
import pandas as pd
from tqdm import tqdm

import torch
from konlpy.tag import Komoran
from pykospacing import Spacing
from transformers import ElectraTokenizer, ElectraModel

komoran = Komoran()
spacing = Spacing()

tqdm.pandas()

In [ ]:
df = pd.read_excel('../../../../../data/filtered_news.xlsx')

# 기본 불용어 불러오기
korean_stopwords_path = '../../../../../data/stopwords-ko.txt'
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
# 띄어쓰기, 대소문자 보정 함수
def preprocessing(text):
    text = spacing(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# komoran토큰화 & 불용어 처리 함수
def remove_stopwords(text, stopwords):
    tokens = []
    morphs = komoran.morphs(text)
    for token in morphs:
        if token not in stopwords:
            tokens.append(token)
    return tokens

# 벡터화 함수
def text_to_vector(text):
    
    # 시퀀스 길이를 같게 하기 위해 패딩 추가  #길이가 긴 시퀀스는 잘라냄  # PyTorch 텐서로 반환
    tokenized_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # 모델에 입력하여 출력 벡터 얻기
    with torch.no_grad():
        outputs = model(**tokenized_inputs)
    cls_vector = outputs.last_hidden_state[0][0].numpy()  # [CLS] 토큰에 대한 벡터 추출
    return cls_vector

In [ ]:
# 문장 전처리 및 형태소 분리, 불용어 처리
cleaned_data = []
for i in range(len(df)):
    feature_text = df.loc[i, 'feature']
    processed_text = preprocessing(feature_text)
    cleaned_text = remove_stopwords(processed_text, stopwords)
    cleaned_data.append(cleaned_text)
df['cleaned'] = cleaned_data

# KoElectra를 통한 벡터화
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
df['vector'] = df['cleaned'].apply(text_to_vector)